In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
from pydantic import BaseModel, FilePath

from rnafold.baseconfig import BaseConfig


class SequenceFiles(BaseModel):
    train: FilePath
    val: FilePath
    test: FilePath


class LabelsFiles(BaseModel):
    train: FilePath
    val: FilePath


class Files(BaseModel):
    sequences: SequenceFiles


class Tools(BaseModel):
    usalign: FilePath


class DataConfig(BaseConfig):
    sequences: SequenceFiles
    labels: LabelsFiles
    submission: FilePath
    tools: Tools


data_config = DataConfig.load_from_yaml("../config/data_local.yml")
data_config

In [ ]:
p = Path("../data/")
files = list(p.glob("*.csv"))
sorted(files)

In [ ]:
for file in sorted(files):
    df = pd.read_csv(file)
    print(file, ":", df.shape)

# Sequences


Columns:

- target_id: unique identifier
- sequence
- temporal_cutoff
- description
- all_sequences


In [ ]:
from rnafold.report import report

report()

In [ ]:
from rnafold.dataset import load_sequences

sequences_train = load_sequences("../data/train_sequences.csv")
sequences_test = load_sequences("../data/test_sequences.csv")

In [ ]:
# Get one sample
ROW = 4

sample = sequences_train.iloc[ROW].to_dict()
sample

In [ ]:
from rnafold.dataset import load_extra_sequences

records = load_extra_sequences(fasta_string=sample["all_sequences"])
records

# Submission


In [ ]:
submission = pd.read_csv("../data/sample_submission.csv")
submission

- there are NaN values in the train dataset -> to investigate


In [ ]:
from rnafold.dataset import load_labels

df = load_labels("../data/train_labels.csv")

# Include N rows before and after
N = 0

# Identify rows with NaN
nan_indices = df.index[df.isna().any(axis=1)]

# Get the range of indices to keep
keep_indices = set()
for idx in nan_indices:
    for offset in range(-N, N + 1):
        new_idx = idx + offset
        if 0 <= new_idx < len(df):  # Ensure within bounds
            keep_indices.add(new_idx)

# Extract the sub-dataframe
sub_df = df.loc[sorted(keep_indices)].reset_index(drop=True)

print(sub_df)
# sub_df.to_csv("nan_in_train.csv")